# Raster Analysis

## Healthy Raster of One Simulation
#### Import Packages


In [ ]:
import numpy as np              # computing tools
import matplotlib.pyplot as plt # plotting 
import scipy.stats as st        # statistics

#### Load in spike data

In [ ]:
SPIKES = np.load('../Data/Healthy/SPIKES_Seed1000.npy',allow_pickle=True)

#### Explore Population Spikelist Structure

In [ ]:
# what's in spikes? 

print(SPIKES.item(0).keys(),'\n') 

In [ ]:
# How many cells in each population? 
popnames = ['Pyr','SST','PV','VIP'] # order of pops

for i in range(4): # number of pops
    print(popnames[i],len(SPIKES.item(0)['times'][i]))

In [ ]:
# Spike times of cell x

'''

# Structure

[[Pyr1,Pyr2,Pyr3....Pyrn],
 [SST1,SST2,SST3....SSTn],
 [PV1,PV2,PV3....PVn],
 [VIP1,VIP2,VIP3,....VIPn]]

'''

celln = 0                               # cell number, try 0 vs 2
print('Pyr cell', 
      SPIKES.item(0)['gids'][0][celln], # cell GID
      [round(i) for i in SPIKES.item(0)['times'][0][celln]]) # spike times in ms

#### Plot Raster

In [ ]:
fig, ax = plt.subplots(figsize=(14,9),ncols=1,nrows=1)   # figure object

pop_colors = ['black','red','green','orange']            # color fo each pop
popnames = ['Pyr','SST','PV','VIP']                      # name for each pop (labels)
fs = 14                                                  # fontsize for plot

for color,spts,gids in zip(pop_colors,SPIKES.item(0)['times'], SPIKES.item(0)['gids']): # for each population...
        t = [] # x
        g = [] # y
        for spt, gid in zip(spts, gids):                              # for each cell
            t = np.r_[t, spt]                                         # add cell spiketime to t
            g = np.r_[g, np.zeros(spt.size)+gid]                      # add cell gid to g
        ax.plot(t[t >= 0], g[t >= 0], 's', color=color, markersize=1) # plot t and g
        
# Formatting
ax.set_xlim(2000,4000)
ax.set_ylim(0,1000)
ax.set_ylabel('Cell Index',fontsize=fs)
ax.set_xlabel('Time (ms)',fontsize=fs)
ax.set_title('Raster Plot of Healthy Simulation',y=1.05,fontsize=18)
ax.tick_params(axis='both', which='major', labelsize=fs)
_ = [ax.spines[side].set_visible(False) for side in ['right','top']]

### Population spike counts over time

#### Subplot A - Raster (same as above)

#### Subplot B - Population spike counts over time

In [ ]:
spikecounts = [0,0,0,0]      # array to hold values
binsize = 10                 # binsize for spike histogram in ms

for z,(pop,pop_spikelist) in enumerate(zip(SPIKES.item(0)['gids'],SPIKES.item(0)['times'])):
    print('Running',popnames[z],'population ...')
    popspikes = np.concatenate(pop_spikelist).ravel() # creates one vector with all spiketimes
    hist,bins = np.histogram(popspikes,bins=np.arange(2000,30000+binsize,binsize)) # remove transient
    hist_max = max(hist)
    spikecounts[z] = [i/hist_max for i in hist] # normalize list for all pops 
    
print('\nRan 4 populations, binned spike vector into',len(bins),'bins')

#### Subplot C - Spikerates of Nonsilent Cells

In [ ]:
spikerates = [0,0,0,0]   # hold valaues

for z,(pop,pop_spikelist) in enumerate(zip(SPIKES.item(0)['gids'],SPIKES.item(0)['times'])):
    print('Running',popnames[z],'population ...')
    nonsilent_count = 0                                                        # count number of nonsilent cells
    popspikes = []                                                             # create empty vector to hold spikes
    for cell_spikes in pop_spikelist:                                          # for each cell spikelist in popspikelist
        cell_spikes_posttrans = [spike for spike in cell_spikes if spike>2000] # remove transient
        cell_spikerate = len(cell_spikes_posttrans)/28                         # spikerate post transient
        if cell_spikerate > .2:                                                # if nonsilent
            popspikes.append(cell_spikerate)                                   # append to population spikelist
            nonsilent_count+=1                                                 # count as nonsilent
    print(nonsilent_count,'nonsilent cells\n')
    spikerates[z]=popspikes


In [ ]:
for i in range(4):
    print(popnames[i],[round(i,3) for i in spikerates[i][:3]]) # spikerates of the first 3 cells in each pop

In [ ]:
spikerates_mean = [np.mean(popspikes) for popspikes in spikerates]
spikerates_sd = [np.std(popspikes) for popspikes in spikerates]

In [ ]:
for z in range(4):
    print('Mean',popnames[z],'cell spikerate:',round(spikerates_mean[z],2),'+/-',round(spikerates_sd[z],2),'Hz')

#### Plot Figure 1

In [ ]:
fig = plt.figure(figsize=(10,8)) # fig object

# controls
xmin,xmax = 2000,4000


#======================
# Plot Raster
#======================
ax = fig.add_axes([0,.38,.65,.58]) # subplot A, to plot raster

for color,spts,gids in zip(pop_colors,SPIKES.item(0)['times'], SPIKES.item(0)['gids']): # same as above
        t = []
        g = []
        for spt, gid in zip(spts, gids):
            t = np.r_[t, spt]
            g = np.r_[g, np.zeros(spt.size)+gid]
        ax.plot(t[t >= 0], g[t >= 0], 's', color=color, markersize=1)

#======================
# Plot Population Spikerates
#======================

x = bins[1:]                                # bins for histogram we created, shared for all axes
VIP = fig.add_axes([0,.24,.65,.06])         # subplot to plot VIP spike counts
VIP.plot(x,spikecounts[3],color='orange')   # spikecounts for VIP

PV = fig.add_axes([0,.16,.65,.06])
PV.plot(x,spikecounts[2],color='green')

SST = fig.add_axes([0,.08,.65,.06])
SST.plot(x,spikecounts[1],color='red')

Pyr = fig.add_axes([0,0,.65,.06])
Pyr.plot(x,spikecounts[0],color='k')


#======================
# Format
#======================

axes = [ax,VIP,PV,SST,Pyr]
titlefs = 16
ax.set_ylim(0,1000)
ax.set_ylabel('Cell Index',fontsize=fs)
ax.set_title('A',loc='left',x=-.1,y=1.03,fontsize=titlefs,weight='bold')
[ax.set_xlim(xmin,xmax) for ax in axes]
[ax.set_ylim(0,1) for ax in axes[1:]]
_ = [ax.spines[side].set_visible(False) for side in ['right','top'] for ax in axes]
[ax.tick_params(axis='both', which='major', labelsize=fs) for ax in axes]
[ax.set_xticklabels([]) for ax in axes[:-1]]
[ax.set_yticklabels([]) for ax in [VIP,SST,PV]]
ax3 = fig.add_axes([.75,.38,.25,.58])
ax3.bar(np.arange(0,4,1),spikerates_mean,yerr=spikerates_sd,capsize=2,color = pop_colors,alpha=.7)
ax3.set_xticks(ticks=[0,1,2,3])
_ = ax3.set_xticklabels(popnames,fontsize=fs)
_ = [ax3.spines[side].set_visible(False) for side in ['right','top']]
ax3.set_ylabel('Spikerate (Hz)',fontsize=fs)
ax3.set_xlabel('Population',fontsize=fs)
ax3.tick_params(axis='both', which='major', labelsize=fs)
ax3.set_title('C',loc='left',x=-.1,y=1.03,fontsize=titlefs,weight='bold')
_=fig.text(x=-.045,y=.02,s='Relative Spike Count',fontsize=fs,rotation=90)
VIP.set_title('B',loc='left',x=-.1,fontsize=titlefs,weight='bold')
Pyr.set_xlabel('Time (ms)',fontsize=fs)




## Healthy Condition Statistics

#### Load in multiple simulations

In [ ]:
# Load in data
SPIKES_H = []                                              # holds spikes from 5 simulations

print('Loading spike data from:\n')

# iterate through filenames
for i in range(1000,1005,1):                               # iterate through 5 files
    filename = '../Data/Healthy/SPIKES_Seed'+str(i)+'.npy' # filename increases with i
    print(filename)                                        # print filename
    SPIKES_H.append(np.load(filename,allow_pickle=True))   # append array of spikes to emptpy SPIKES list
    

#### Spike Rates of Nonsilent Cells

In [ ]:
# Creat vector to hold spikes
spikerates_h = [[0,0,0,0] for i in range(len(SPIKES_H))] # 5 (runs) x 4 (pops/run)
spikerates_h

In [ ]:
# Create population spikerate vector of nonsilent cells for each run

spikerates_h = [[0,0,0,0] for i in range(len(SPIKES_H))]             # 5 (runs) x 4 (pops/run)

for r,SPIKES in enumerate(SPIKES_H):                                 # for each run in the healthy condition
    print('Processing run',r,'...')                                  # r = spikesSEED#
    for z,(pop,pop_spikelist) in enumerate(zip(SPIKES.item(0)['gids'],SPIKES.item(0)['times'])): # popname, popids, spikes
        popspikes = []                                               # empty list to all all nonsilent spikes
        for cell_spikes in pop_spikelist:                            # each cell spikelist in popspikelist
            cell_spikes_posttrans = [spike for spike in cell_spikes if spike>2000] # only take spikes after 2000ms
            cell_spikerate = len(cell_spikes_posttrans)/28           # spikerate = # spikes in list / 28 s
            if cell_spikerate > .2:                                  # .2 Hz threshold
                popspikes.append(cell_spikerate)                     # add cell spikerate to list
        spikerates_h[r][z]=popspikes                                 # put spikerates in run x pop

In [ ]:
# View spike rates of first 3 cells in pop

run = 0
pop = 0

print('All Healthy run',run,'pop',pop,
      [round(i,2) for i in spikerates_h[run][pop][:3]]) # 5 runs x 4 pops x poplen spikerates

if run ==0:
    print('First Healthy          ',[round(i,2) for i in spikerates[pop][:3]])

In [ ]:
# Calculate mean population spike rates for each run

spikerates_h_mean_temp = [] # store spikerates

for run in spikerates_h:
    spikerates_h_mean_temp.append([np.mean(popspikes) for popspikes in run]) # [Pyr,SST,PV,VIP]
    
spikerates_h_mean_temp = np.array(spikerates_h_mean_temp)

In [ ]:
# View population spike rates for each run

spikerates_h_mean_temp

In [ ]:
# Calculate mean population spike rates for each pop

spikerates_h_mean = np.mean(spikerates_h_mean_temp,axis=0)
spikerates_h_sd = np.std(spikerates_h_mean_temp,axis=0)

spikerates_h_mean

In [ ]:
fig = plt.figure(figsize=(8,5))      # fig obj

#======================
# Spikerates
#======================
axes = plt.subplot(111)               # subplot for plotting
x = np.arange(0,4,1)                  # 4 populations, so 4 ticks [0,1,2,3]

axes.bar(x = x,                       # place bars
         height=spikerates_h_mean,    # heigh of bar
         yerr=spikerates_h_sd,        # SD
         color=pop_colors,            # color code pops
         alpha=.6,                    # decrease opacity to better see error bar
         capsize=3)                   # error bar capsize



#======================
# Formatting
#======================
axes.set_xticks(ticks=[0,1,2,3])
axes.set_xticklabels(popnames,fontsize=fs)
[axes.spines[side].set_visible(False) for side in ['right','top']]
axes.set_xlabel('Population',fontsize=fs)
axes.tick_params(axis='both', which='major', labelsize=fs)
axes.set_ylabel('Spikerate (Hz)',fontsize=fs)
axes.set_title('Population Spike Rate',y=1.05,fontsize=titlefs)
plt.tight_layout()

## Compare Healthy and Depression Microcircuits

### Task 1: Compare Raster Plots between Conditions

In [ ]:
# Load in Depression data
print('Depression Spikes:')


In [ ]:
# Plot Raster of Healthy and Depression
fig = plt.figure(figsize=(15,8)) # figure object

# Controls

run = 0
xmin,xmax = 2000,4000

#======================
# Healthy
#======================

healthy = plt.subplot(121) # subplot to hold healthy raster
for color,spts,gids in zip(pop_colors,SPIKES_H[run].item(0)['times'], SPIKES_H[run].item(0)['gids']):
        t = []
        g = []
        for spt, gid in zip(spts, gids):
            t = np.r_[t, spt]
            g = np.r_[g, np.zeros(spt.size)+gid]
        healthy.plot(t[t >= 0], g[t >= 0], 's', color=color, markersize=1)


#======================
# Depression
#======================
# use spikes from one depression simulation, and plot

depression = plt.subplot(122) # subplot to hold depression raster

        
#======================
# Formatting
#======================
axes = [healthy,depression]
healthy.set_ylabel('Cell Index',fontsize=fs)
healthy.set_title('Raster Plot of Healthy Simulation',y=1.05,fontsize=titlefs)
depression.set_title('Raster Plot of Depression Simulation',y=1.05,fontsize=titlefs)
[ax.set_xlim(xmin,xmax) for ax in axes]
[ax.set_ylim(0,1000) for ax in axes]
[ax.set_xlabel('Time (ms)',fontsize=fs) for ax in axes]
[ax.tick_params(axis='both', which='major', labelsize=fs) for ax in axes]
[ax.spines[side].set_visible(False) for side in ['right','top'] for ax in axes]
[ax.set_title('Raster Plot of Depression Simulation',y=1.05,fontsize=titlefs)]
plt.tight_layout()

### Task 2: Compare Spike Rates Between Conditions

In [ ]:
# Creat vector to hold spikes


In [ ]:
# Create population spike vector of nonsilent cells for each run


In [ ]:
# Calculate mean population spike rates for each run



In [ ]:
# View population spike rates for each run



In [ ]:
# Calculate mean population spike rates for each pop



In [ ]:
#======================
# Plot Spikerates
#======================

#======================
# Setup and Parameters --- do not change
#======================
fig = plt.subplots(figsize=(12,6)) # figure obj

dclr = (0.8, 0.0, 0.45)            # RBG value for depression
d_colors = [dclr,dclr,dclr,dclr]   # list of clolors for population bar depression

x = np.arange(0,4,1)               # xticks for population rates

width = 0.4                        # width of bars

h_offset = 0.0                     # use same xtick for h and d, so set offset to 0 for h
d_offset = 0.4                     # shift depression bar .4 to right

pyr_rates = plt.subplot(121)       # subplot for pyramidal rates (zoom in)
population_rates = plt.subplot(122)# subplot for all population rates


### Healthy
# Population spike rates
population_rates.bar(x+h_offset,spikerates_h_mean,width=width,yerr=spikerates_h_sd,color=pop_colors,alpha=.6,capsize=3)

# Pyramidal rates zoom in
pyr_rates.bar(x[0]+h_offset,spikerates_h_mean[0],width=width,yerr=spikerates_h_sd[0],color=pop_colors[0],alpha=.6,capsize=3)

### Depression
# use mean spike rates and sd spike rates
# Population rates zoom in

# Pyramidal rates zoom in
# use mean spike rate and sd from Pyramidals


#======================
# Formatting
#======================
pyr_rates.set_title('Pyramidal Spike Rate',y=1.05,fontsize=titlefs)
pyr_rates.set_ylabel('Spikerate (Hz)',fontsize=fs)
pyr_rates.set_xlabel('Condition',fontsize=fs)
pyr_rates.set_xticks(ticks=[0,.4])
pyr_rates.set_xlim(-.5,1)
pyr_rates.set_xticklabels(['Healthy','Depression'],fontsize=fs)
pyr_rates.set_ylim(0,1.7)

population_rates.set_title('Population Spike Rates',y=1.05,fontsize=titlefs)
population_rates.set_ylabel('Spikerate (Hz)',fontsize=fs)
population_rates.set_xlabel('Population',fontsize=fs)
population_rates.set_xticks(ticks=[0,1,2,3])
population_rates.set_xticklabels(popnames,fontsize=fs)

axes = [pyr_rates,population_rates]
[ax.spines[side].set_visible(False) for side in ['right','top'] for ax in axes]
[ax.tick_params(axis='both', which='major', labelsize=fs) for ax in axes]
[ax.legend(['Healthy','Depression'],loc='upper right',bbox_to_anchor=(1.1,1)) for ax in axes]
plt.tight_layout()


### Task 3: Calculate Difference, Significance, and Effect Size (If time permits)

#### Load Necessary Packages and Functions 

In [ ]:
def cohen_d(mean1,std1,nobs1,mean2,std2,nobs2):        # effect size
    nx = nobs1
    ny = nobs2
    dof = nx + ny - 2
    return (mean1 - mean2) / np.sqrt(((nx-1)*std1 ** 2 + (ny-1)*std2 ** 2) / dof)

#### Difference

In [ ]:
# VIP Example

VIP_diff = (spikerates_d_mean[3]-spikerates_h_mean[3])*100
print('There is a',round(VIP_diff,1),'% increase in VIP spikerate')

In [ ]:
# other populations


#### P-value

In [ ]:
pvals = [1,1,1,1]

# VIP Example
_,pvals[0] = st.ttest_ind_from_stats(
                        mean1 = spikerates_h_mean[3], # Health VIP mean
                        std1 = spikerates_h_sd[3],    # Healthy VIP sd
                        nobs1 = 5,                    # number of simulations included in analysis per condition
                        mean2 = spikerates_d_mean[3], # Depression VIP mean
                        std2 = spikerates_d_sd[3],    # Depression VIP sd
                        nobs2 = 5)                    # number of simulations included in analysis per condition


In [ ]:
# other populations
pvals = [1,1,1,1]


#### Effect Size

In [ ]:
# VIP Example
dvals = [0,0,0,0]
dof = len(SPIKES_H)+len(SPIKES_D)-2
VIP_d = cohen_d(mean1 = spikerates_h_mean[3], # Health VIP mean
                std1 = spikerates_h_sd[3],    # Healthy VIP sd
                nobs1 = 5,                    # number of simulations included in analysis per condition
                mean2 = spikerates_d_mean[3], # Depression VIP mean
                std2 = spikerates_d_sd[3],    # Depression VIP sd
                nobs2 = 5)   

print('Effect size for diff in VIP spikerates is',round(VIP_d,2))